<a href="https://colab.research.google.com/github/Liang130520/Tibame-TeamProject/blob/main/Catboost%E9%A9%97%E8%AD%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
from pandas.core.dtypes.cast import maybe_infer_to_datetimelike
"""
使用方式：
1. 點選上方執行階段 -> 全部執行
2. 允許連接雲端
3. 輸入名字後兩字
4. 將自己名字資料夾中的r2_score最高分數檔案留下，其餘刪除
5. 將r2_score最高分數的檔案名稱(EX：台北市房屋交易_公園100公尺.csv)複製
6. 在「特徵-公尺內設施數量」程式的第四格程式碼中的str(name)+"/後面貼上

"""

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import glob
import os
from catboost import CatBoostRegressor

from google.colab import drive
drive.mount('/content/drive')

drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/程式語言/特徵檔案') #切換該目錄
os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['raw_data',
 '宗庭',
 '博翔',
 '俊佑',
 '聖評',
 '柏崴',
 '冠文',
 '設施座標表.xlsx',
 '設施座標表Ver2.xlsx',
 '聖評台北市房屋交易初始版Ver1.csv',
 '台北市房屋交易_test.csv',
 '台北市房屋交易_finish.csv',
 '.ipynb_checkpoints',
 'catboost_info']

In [39]:
# 原始檔案

# name = input("請輸入名字後兩字：")
name = "博翔"
f = pd.read_csv(str(name) + "/台北市房屋交易_for_model_1126_加設施_3.csv", encoding = "utf-8")  # 讀取csv檔案

pd.set_option('display.max_columns', 100)

x = f.drop(["土地位置建物門牌","總價元","單價元平方公尺","車位總價元","總價元_正規化","單價元平方公尺_正規化","車位總價元_正規化"], axis = 1)  # 特徵(去除總價元、單價元平方公尺、車位總價元)
y = f["總價元_正規化"]  # 目標(取得總價元)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)  # 切分資料

In [40]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.2, random_state=1234)
categorical_features_indices = np.where(x.dtypes != np.float)[0]

<ipython-input-40-fa3835327f3e>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  categorical_features_indices = np.where(x.dtypes != np.float)[0]


In [41]:
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm

# feature_names = ['F{}'.format(i) for i in range(x_train.shape[1])]
train_pool = Pool(x_train, y_train, feature_names=x.columns.tolist())
test_pool = Pool(x_test, y_test, feature_names=x.columns.tolist())

" 模型選擇 "
model = CatBoostRegressor(random_state=42,
                          loss_function='RMSE',
                          eval_metric='RMSE',
                          learning_rate=0.1,
                          use_best_model=True,
                          depth=3)

" 抓取最有效特徵 "
summary = model.select_features(
    train_pool,
    eval_set=test_pool,
    features_for_select='0-12',
    num_features_to_select=3,
    steps=2,
    algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
    shap_calc_type=EShapCalcType.Regular,
    train_final_model=True,
    logging_level='Silent',
    plot=False
)
summary

{'selected_features': [4, 5, 10],
 'eliminated_features_names': ['車位移轉總面積(平方公尺)',
  '陽台面積',
  '建物現況格局-房',
  '交易年月日',
  '建物現況格局-廳',
  '建物現況格局-衛',
  '土地移轉總面積平方公尺',
  '附屬建物面積',
  '總樓層數',
  '移轉層次'],
 'loss_graph': {'main_indices': [0, 7, 10],
  'removed_features_count': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
  'loss_values': [0.056551505436996545,
   0.05538175622054835,
   0.05508871469934309,
   0.05486824575751033,
   0.054740554996119774,
   0.054685243781109366,
   0.05479424653071746,
   0.05493839753251216,
   0.054907009954912704,
   0.05484897130439829,
   0.054678289024979246]},
 'eliminated_features': [9, 12, 6, 1, 7, 8, 0, 11, 3, 2],
 'selected_features_names': ['建築完成年月', '建物移轉總面積平方公尺', '主建物面積']}

In [42]:
from catboost import CatBoostRegressor

" 模型訓練 "
model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test), verbose=0, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.inspection import permutation_importance

pre = model.predict(x_test)
rmse = (np.sqrt(mean_squared_error(y_test, pre)))
r2 = r2_score(y_test, pre)
print("Testing performance")
print("RMSE: {}".format(rmse))
print("R2: {}".format(r2))
model.get_best_score()

Testing performance
RMSE: 0.05893397271951207
R2: 0.8577931928216231


{'learn': {'RMSE': 0.019804965760522212},
 'validation': {'RMSE': 0.05893397256284466}}